In [ ]:
#|default_exp fetch_coords

# Fetch Coords

> Utilities to fetch json file containing coordinates metadata

In [ ]:
#| eval: false
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import requests
import json
from fastcore.all import L
import pandas as pd
from typing import Any
from urllib.parse import urlparse
from pathlib import Path
import numpy as np
import re
from shapely.geometry import Polygon, box
import geopandas as gpd
import sqlite3 as sql3
from pandas.io import sql 
import sqlalchemy as sqalc

In [ ]:
#| exporti
from ntlights_damage_assessment.build_catalogs import DB
from ntlights_damage_assessment.catalogs import get_data

In [ ]:
#| hide
import matplotlib.pyplot as plt

In [ ]:
#| hide
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',120)

In [ ]:
#| export

def get_coords(href, stem):
    data = get_data(href)
    shape = Polygon(data['geometry']['coordinates'][0])
    asset_href = data['assets']['image']['href']
    row = dict(
        stem=stem,
        asset_href=data['assets']['image']['href'],
        geometry=shape
    )
    return row

In [ ]:
stem = 'SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops'

In [ ]:
href = 'https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops.rade9.co.json'


In [ ]:
%%time
#| eval: false
row = get_coords(href,stem)

CPU times: user 28.8 ms, sys: 9.47 ms, total: 38.3 ms
Wall time: 1.35 s


In [ ]:
row

{'stem': 'SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops',
 'asset_href': 'https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops.rade9.co.tif',
 'geometry': <shapely.geometry.polygon.Polygon>}

In [ ]:
date_param = '20180501'

In [ ]:
if '%' in date_param:
    query = f'select * from catalog_items where start_date like "{date_param}"'
else:
    query = f'select * from catalog_items where start_date = "{date_param}"' 

In [ ]:
#| eval: false
conn = sql3.connect(DB)

In [ ]:
%%time
#| eval: false
items_df = pd.read_sql_query(query,conn,dtype=str)

CPU times: user 35.6 ms, sys: 336 ms, total: 371 ms
Wall time: 401 ms


In [ ]:
items_df.columns.values

array(['href', 'item_href', 'stem', 'suffix', 'base_url', 'folder',
       'product_id', 'start_date', 'first_scantime', 'end_scantime',
       'orbital_nbr', 'create_datetime', 'data_origin', 'data_domain',
       'vflag_file', 'vflag_href'], dtype=object)

In [ ]:
items_df.head()

,href,item_href,stem,suffix,base_url,folder,product_id,start_date,first_scantime,end_scantime,orbital_nbr,create_datetime,data_origin,data_domain,vflag_file,vflag_href
0,./SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522...,SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0004098,0009502,33718,20180501060951522600,noac,ops,npp_d20180501_t0004098_e0009502_b33718.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0004098_e0009502_b33718.vflag.co.tif
1,./SVDNB_npp_d20180501_t0009514_e0015318_b33718_c20180501061532603366_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0009514_e0015318_b33718_c20180501061532603...,SVDNB_npp_d20180501_t0009514_e0015318_b33718_c20180501061532603366_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0009514,0015318,33718,20180501061532603366,noac,ops,npp_d20180501_t0009514_e0015318_b33718.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0009514_e0015318_b33718.vflag.co.tif
2,./SVDNB_npp_d20180501_t0112274_e0118078_b33719_c20180501071808369542_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0112274_e0118078_b33719_c20180501071808369...,SVDNB_npp_d20180501_t0112274_e0118078_b33719_c20180501071808369542_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0112274,0118078,33719,20180501071808369542,noac,ops,npp_d20180501_t0112274_e0118078_b33719.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0112274_e0118078_b33719.vflag.co.tif
3,./SVDNB_npp_d20180501_t0118090_e0123494_b33719_c20180501072349439701_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0118090_e0123494_b33719_c20180501072349439...,SVDNB_npp_d20180501_t0118090_e0123494_b33719_c20180501072349439701_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0118090,0123494,33719,20180501072349439701,noac,ops,npp_d20180501_t0118090_e0123494_b33719.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0118090_e0123494_b33719.vflag.co.tif
4,./SVDNB_npp_d20180501_t0123507_e0129293_b33719_c20180501072930507231_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0123507_e0129293_b33719_c20180501072930507...,SVDNB_npp_d20180501_t0123507_e0129293_b33719_c20180501072930507231_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0123507,0129293,33719,20180501072930507231,noac,ops,npp_d20180501_t0123507_e0129293_b33719.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0123507_e0129293_b33719.vflag.co.tif


In [ ]:
#| export

def get_rows(items_df):
    rows = []
    for i in range(len(items_df)):
        item = items_df.iloc[i]
        row = get_coords(item.item_href, item.stem)
        rows.append(row) 
    return rows

In [ ]:
%%time
#| eval: false
rows = get_rows(items_df)

CPU times: user 3.71 s, sys: 127 ms, total: 3.84 s
Wall time: 2min 37s


In [ ]:
rows_df = pd.DataFrame.from_records(rows)

In [ ]:
rows_df.columns.values

array(['stem', 'asset_href', 'geometry'], dtype=object)

In [ ]:
rows_df.dtypes

stem          object
asset_href    object
geometry      object
dtype: object

In [ ]:
rows_gdf = gpd.GeoDataFrame(rows_df, geometry='geometry', crs='EPSG:4326')

In [ ]:
rows_wkt  = rows_gdf.to_wkt()

In [ ]:
rows_wkt.dtypes

stem          object
asset_href    object
geometry      object
dtype: object

In [ ]:
#| exporti

create_coords_cache_sql = '''
CREATE TABLE IF NOT EXISTS "coords_cache" (
  "stem" TEXT,
  "asset_href" TEXT,
  "geometry" TEXT
);
CREATE INDEX cache_index on coords_cache(stem);
'''
drop_coords_cache_sql = '''
DROP TABLE IF EXISTS coords_cache;
'''


In [ ]:
# conn.executescript(drop_coords_cache_sql);

<sqlite3.Cursor>

In [ ]:
# conn.executescript(create_coords_cache_sql);

In [ ]:
%%time
#| eval: false

row_count = rows_wkt.to_sql('coords_cache', conn, if_exists='append', index=False, method='multi')

CPU times: user 0 ns, sys: 3.99 ms, total: 3.99 ms
Wall time: 13 ms


In [ ]:
#| export

def add_cache_coords(items_df, conn, cache_table='coords_cache'):
    rows = get_rows(items_df)
    rows_df = pd.DataFrame.from_records(rows)
    rows_gdf = gpd.GeoDataFrame(rows_df, geometry='geometry', crs='EPSG:4326')
    rows_wkt  = rows_gdf.to_wkt()
    row_count = rows_wkt.to_sql(cache_table, conn, if_exists='append', index=False, method='multi')
    return row_count

In [ ]:
joined_query_template = '''
with item as ( 
  select *
  from catalog_items
),
cache as (
  select *
  from coords_cache
),
item_cache as (
  select item.*,
         ifnull(cache.stem,'NA') as cache_stem,
         cache.asset_href,
         cache.geometry
  from item
  left join cache
  on item.stem = cache.stem
)
select * from item_cache
where {}
'''
  

In [ ]:
len(joined_df)

120

In [ ]:
def fetch_cached_coordinates(date_param):
    if '%' in date_param:
        criteria = f'start_date like "{date_param}"'
    else:
        criteria = f'start_date = "{date_param}"' 
    joined_query = joined_query_template.format(criteria)

    conn = sql3.connect(DB)
    joined_df = pd.read_sql_query(joined_query, conn, dtype=str)
    conn.close()
    return joined_df

In [ ]:
%%time
#| eval: false
cached_df = fetch_cached_coordinates('20180501')

CPU times: user 47.9 ms, sys: 139 ms, total: 187 ms
Wall time: 202 ms


In [ ]:
%%time
#| eval: false
cached_df2 = fetch_cached_coordinates('20180515')

CPU times: user 48.8 ms, sys: 109 ms, total: 158 ms
Wall time: 170 ms


In [ ]:
%%time
#| eval: false
items_df = pd.read_sql_query(query,conn,dtype=str)

CPU times: user 35.6 ms, sys: 336 ms, total: 371 ms
Wall time: 401 ms


In [ ]:
items_df.columns.values

array(['href', 'item_href', 'stem', 'suffix', 'base_url', 'folder',
       'product_id', 'start_date', 'first_scantime', 'end_scantime',
       'orbital_nbr', 'create_datetime', 'data_origin', 'data_domain',
       'vflag_file', 'vflag_href'], dtype=object)

In [ ]:
items_df.head()

,href,item_href,stem,suffix,base_url,folder,product_id,start_date,first_scantime,end_scantime,orbital_nbr,create_datetime,data_origin,data_domain,vflag_file,vflag_href
0,./SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522...,SVDNB_npp_d20180501_t0004098_e0009502_b33718_c20180501060951522600_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0004098,0009502,33718,20180501060951522600,noac,ops,npp_d20180501_t0004098_e0009502_b33718.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0004098_e0009502_b33718.vflag.co.tif
1,./SVDNB_npp_d20180501_t0009514_e0015318_b33718_c20180501061532603366_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0009514_e0015318_b33718_c20180501061532603...,SVDNB_npp_d20180501_t0009514_e0015318_b33718_c20180501061532603366_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0009514,0015318,33718,20180501061532603366,noac,ops,npp_d20180501_t0009514_e0015318_b33718.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0009514_e0015318_b33718.vflag.co.tif
2,./SVDNB_npp_d20180501_t0112274_e0118078_b33719_c20180501071808369542_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0112274_e0118078_b33719_c20180501071808369...,SVDNB_npp_d20180501_t0112274_e0118078_b33719_c20180501071808369542_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0112274,0118078,33719,20180501071808369542,noac,ops,npp_d20180501_t0112274_e0118078_b33719.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0112274_e0118078_b33719.vflag.co.tif
3,./SVDNB_npp_d20180501_t0118090_e0123494_b33719_c20180501072349439701_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0118090_e0123494_b33719_c20180501072349439...,SVDNB_npp_d20180501_t0118090_e0123494_b33719_c20180501072349439701_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0118090,0123494,33719,20180501072349439701,noac,ops,npp_d20180501_t0118090_e0123494_b33719.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0118090_e0123494_b33719.vflag.co.tif
4,./SVDNB_npp_d20180501_t0123507_e0129293_b33719_c20180501072930507231_noac_ops.rade9.co.json,https://globalnightlight.s3.amazonaws.com/npp_201805/SVDNB_npp_d20180501_t0123507_e0129293_b33719_c20180501072930507...,SVDNB_npp_d20180501_t0123507_e0129293_b33719_c20180501072930507231_noac_ops,rade9.co.json,https://globalnightlight.s3.amazonaws.com,npp_201805,SVDNB_npp,20180501,0123507,0129293,33719,20180501072930507231,noac,ops,npp_d20180501_t0123507_e0129293_b33719.vflag.co.tif,https://globalnightlight.s3.amazonaws.com/npp_201805/npp_d20180501_t0123507_e0129293_b33719.vflag.co.tif
